In [1]:
import pandas as pd
import numpy as np

In [2]:
path_to_slots_data = '/Users/tonny/Desktop/DS/DSP/Project Material/Datasets/export-3-nov/fct_slot_interaction'

listpages = pd.read_parquet('./data/fct_list_pages.parque')
listpages['containerId'] = listpages['eventId'].map(lambda e: e.split('.')[0])
pageview = pd.read_parquet('./data/fct_page_view_nl.parque')
slots = pd.read_parquet(path_to_slots_data)

In [3]:
slots = slots.convert_dtypes()
slots['eventTimestamp'] = pd.to_datetime(slots['eventTimestamp'])
slots.dtypes

visitId           string[python]
eventTimestamp    datetime64[us]
groupId           string[python]
pageName          string[python]
groupType         string[python]
groupName         string[python]
groupTitle        string[python]
hasView                  boolean
hasClick                 boolean
isWebsite                  Int64
isAndroid                  Int64
isIOS                      Int64
dtype: object

In [4]:
pageview['containerId'] = pageview['eventId']
pageview_min = pageview[['containerId', 'pageName']]
listpages_named = pd.merge_ordered(how='inner', left=listpages, on='containerId', right=pageview_min)
listpages_named.shape

(212032, 14)

In [5]:
np.unique(slots['pageName'].tolist())

array(['ACCOUNT', 'ADDONPAGE', 'BRAND', 'CUSTOMERSERVICE', 'EXPLORE',
       'HOME', 'OHP', 'PDP', 'RA', 'SEARCH', 'UNKNOWN'], dtype='<U15')

In [6]:
splits = pageview['hookId'].str.extract(r'^(.*?)\.(.*)\.(.*?)$')

In [7]:
pageview['eventId'] = splits[0]
pageview['itemId'] = splits[1]
pageview['callToAction'] = splits[2]

In [8]:
len(np.unique(pageview['visitId'].tolist()))

134047

In [9]:
added_cart = pageview.loc[pageview['callToAction'] == 'AddToCart']
payment = pageview.loc[pageview['pageName'] == 'PaymentDone']

visit_id = np.unique(added_cart['visitId'].to_list())
visit_id2 = np.unique(payment['visitId'].to_list())

len(visit_id2) *100 / len(visit_id)

83.08148566586641

In [10]:
len(np.unique(visit_id2))

5883

In [11]:
#all visitIDs where an item is put into cart and bought in the same visitID
bought_session = list(set(visit_id2).intersection(visit_id))

In [12]:
test = bought_session[777]

tmp = slots.loc[slots['visitId'] == test]
tmp = tmp.sort_values('eventTimestamp', ascending=True)
tmp2 = added_cart.loc[added_cart['visitId'] == test]

In [13]:
concatenated_df = pd.concat([tmp, tmp2], axis=0)

concatenated_df

,visitId,eventTimestamp,groupId,pageName,groupType,groupName,groupTitle,hasView,hasClick,isWebsite,isAndroid,isIOS,_surrogate_key,dateAmsterdam,eventId,hookId,containerId,itemId,callToAction
3738096,QIo2N0gHkllvDUBnX4Gh5IYP7ITD+k0aVxmzWoVIt3c,2022-11-07 09:49:42.124,hbtpJvF6piTDCLRaSRlHkA.56_57,HOME,DayDeal,List,Dagdeal,False,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2951855,QIo2N0gHkllvDUBnX4Gh5IYP7ITD+k0aVxmzWoVIt3c,2022-11-07 09:49:42.124,hbtpJvF6piTDCLRaSRlHkA.5_14,HOME,Flexbanner,Flexbanner,<NA>,True,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
707917,QIo2N0gHkllvDUBnX4Gh5IYP7ITD+k0aVxmzWoVIt3c,2022-11-07 09:49:42.124,hbtpJvF6piTDCLRaSRlHkA.5_16,HOME,Flexbanner,Flexbanner,<NA>,True,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2577147,QIo2N0gHkllvDUBnX4Gh5IYP7ITD+k0aVxmzWoVIt3c,2022-11-07 09:49:42.124,hbtpJvF6piTDCLRaSRlHkA.2_3,HOME,Topbanner,Topbanner,<NA>,True,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2095610,QIo2N0gHkllvDUBnX4Gh5IYP7ITD+k0aVxmzWoVIt3c,2022-11-07 09:49:42.124,hbtpJvF6piTDCLRaSRlHkA.56_59,HOME,Flexbanner,Flexbanner,<NA>,False,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7060984,QIo2N0gHkllvDUBnX4Gh5IYP7ITD+k0aVxmzWoVIt3c,2022-11-07 09:49:42.124,hbtpJvF6piTDCLRaSRlHkA.5_8,HOME,Flexbanner,Flexbanner,<NA>,True,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1083282,QIo2N0gHkllvDUBnX4Gh5IYP7ITD+k0aVxmzWoVIt3c,2022-11-07 09:49:42.124,hbtpJvF6piTDCLRaSRlHkA.5_12,HOME,Flexbanner,Flexbanner,<NA>,True,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8378689,QIo2N0gHkllvDUBnX4Gh5IYP7ITD+k0aVxmzWoVIt3c,2022-11-07 09:49:57.750,jKI6qbXCIHFwgwGR0N9Fow.2_9,PDP,product-images,product-images,product-images,True,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4420044,QIo2N0gHkllvDUBnX4Gh5IYP7ITD+k0aVxmzWoVIt3c,2022-11-07 09:49:57.750,jKI6qbXCIHFwgwGR0N9Fow.2_9_14,PDP,product-image-thumbs,product-image-thumbs,product-image-thumbs,False,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4397286,QIo2N0gHkllvDUBnX4Gh5IYP7ITD+k0aVxmzWoVIt3c,2022-11-07 09:49:57.750,jKI6qbXCIHFwgwGR0N9Fow.2_23,PDP,ProductDescription,ProductDescription,Productbeschrijving,False,False,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
page_sequence = concatenated_df['groupTitle'].dropna().tolist()
page_sequence

['Dagdeal',
 'product-images',
 'product-image-thumbs',
 'Productbeschrijving',
 'Vaak samen gekocht',
 'ReviewItems',
 'Anderen bekeken ook',
 'Reviews',
 'Productspecificaties',
 'Recent bezochte artikelen',
 'Recent bezochte artikelen',
 'Verfijn je resultaten',
 'Recent bezochte artikelen',
 'Recent bezochte artikelen',
 'Dagdeal',
 'Recent bezochte artikelen',
 'Dagdeal',
 'Bundel met accessoire',
 'Productbeschrijving',
 'Productspecificaties',
 'ReviewItems',
 'Vaak samen gekocht',
 'Bundel met batterijen.',
 'Bundel met batterijen.',
 'Reviews',
 'Anderen bekeken ook',
 'product-image-thumbs',
 'product-images',
 'Bekijk ook eens',
 'Batterijen',
 'Diverse accessoires',
 'Populaire categorieën',
 'Populaire categorieën']

In [19]:
def get_sequence(visit_id):
    visit_slots = slots[slots['visitId'] == visit_id].sort_values('eventTimestamp')
    visit_added_cart = added_cart[added_cart['visitId'] == visit_id]
    concatenated_df = pd.concat([visit_slots, visit_added_cart], axis=0)
    
    return concatenated_df['groupTitle'].dropna().tolist()


In [25]:
len(bought_session)

3380

In [24]:
# features = {}
# for i in bought_session:
#     features[i] = get_sequence(i)

KeyboardInterrupt: 

In [21]:
features

{'kkvTHHnHBQR7NylwwM+emetMOVZpQKjrS5OpE0oSog0': 0,
 'ivARhGS0j+mOgbG2RFWyaqxCHgGrS8kTgYsaOQSjhEs': 0,
 'WJVyAGErrT2bT3mHaROuFPIwpVPNn/TTd3OhW0m8dVY': 0,
 'cIO8Q/4x4fG4CM1WpXQ+FoVmySZiHeZWXLXvmlE5JDc': 0,
 'AZ5KYrVDGX5kIK6fHZuj/esfKeg8zOtoQtn5tfLqt6M': 0,
 'qo1fCvHJ6bXAN3cbFXk9EPP6yPGBVGQS1kBSGb+4kGI': 0,
 'ry86q2s5rR+UyG/rLXwn86+/z4rRzyir14PazYNB0Sk': 0,
 'uyAJCkCOpWtMb27dwDbTF1MsRrV+EM/H7PxPyTNiBWc': 0,
 'Dvn/qhMZn969+Y5RJihAsiZ6tY+LB0s0CjnsoUGdDhQ': 0,
 '2Z+C6vFgL1Lf1/zcF4X+RhTEpIz3IAUITl4kO7AySqE': 0,
 'rwG06n+regJ2iNEWMt2w3uRyaR1rN9ognVkTUW7c59g': 0,
 'XSK2TFpVkM6RLceEiD2nI8Jx2mXrICrC6vumMIkQSLI': 0,
 'rL3IOEnPUHuwFTt/iA4nm84f5Ev91LcMuH7F/5F4xUI': 0,
 '+mk8TKr6OK+FqsHS9lipaea4MVJ8jQAqKw5MylqAOic': 0,
 'M3SDRQugaGvi4wp3M7wBDpDF6HykHvn9pVWHPkgjtn4': 0,
 'fv6SwZABZNMXQjq5kWGgSBt+OY0JuYckZ7Yp7DMNJbw': 0,
 'Wre2UtjE0tRSEjeIpR79A5xmvUxfSwxu73Qn64/tfOg': 0,
 'iOwMoqLGG6379AxPtZC4AxOsS21n+YYIIt0tBKXcgPU': 0,
 'gOa2mwLh5VQmrEIUvSxeSdPz2leXVWN7Pcl/oTtMNhQ': 0,
 'SjZE0w+yF1Is8SA8+6cbTSv+rWS7p

LIKELINESS SCORE